<a href="https://colab.research.google.com/github/ryan-ribeiro/lia1_2024_2/blob/main/Entregas%20-%20Ryan%20Ribeiro/traffic-signs-yolov5/YOLOv5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv5 Classification Tutorial

YOLOv5 supports classification tasks too. This is the official YOLOv5 classification notebook tutorial. YOLOv5 is maintained by [Ultralytics](https://github.com/ultralytics/yolov5).

This notebook covers:

*   Inference with out-of-the-box YOLOv5 classification on ImageNet
*  [Training YOLOv5 classification](https://blog.roboflow.com//train-YOLOv5-classification-custom-data) on custom data

*Looking for custom data? Explore over 66M community datasets on [Roboflow Universe](https://universe.roboflow.com).*

This notebook was created with Google Colab. [Click here](https://colab.research.google.com/drive/1FiSNz9f_nT8aFtDEU3iDAQKlPT8SCVni?usp=sharing) to run it.

# Setup

Pull in respective libraries to prepare the notebook environment.

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-377-g24ee2801 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.3/112.6 GB disk)


# 1. Infer on ImageNet

To demonstrate YOLOv5 classification, we'll leverage an already trained model. In this case, we'll download the ImageNet trained models pretrained on ImageNet using YOLOv5 Utils.

In [ ]:
from utils.downloads import attempt_download

p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

100%|██████████| 4.87M/4.87M [00:00<00:00, 80.3MB/s]

100%|██████████| 10.5M/10.5M [00:00<00:00, 85.2MB/s]

100%|██████████| 24.9M/24.9M [00:00<00:00, 107MB/s] 

100%|██████████| 50.9M/50.9M [00:00<00:00, 109MB/s]

100%|██████████| 92.0M/92.0M [00:00<00:00, 104MB/s]



Now, we can infer on an example image from the ImageNet dataset.

In [ ]:
#Download example image
import requests
image_url = "https://i.imgur.com/OczPfaz.jpg"
img_data = requests.get(image_url).content
with open('bananas.jpg', 'wb') as handler:
    handler.write(img_data)

In [ ]:
#Infer using classify/predict.py
!python classify/predict.py --weights ./weigths/yolov5s-cls.pt --source bananas.jpg

classify/predict: weights=['./weigths/yolov5s-cls.pt'], source=bananas.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-377-g24ee2801 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

100% 10.5M/10.5M [00:00<00:00, 92.4MB/s]

Fusing layers... 
Model summary: 117 layers, 5447688 parameters, 0 gradients, 11.4 GFLOPs
Traceback (most recent call last):
  File "/content/yolov5/classify/predict.py", line 241, in <module>
    main(opt)
  File "/content/yolov5/classify/predict.py", line 236, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov5/classify/predict.py", line 122, in run
    for path, im, im0s, vid_cap, s in dataset:
  File "/content/yo

From the output, we can see the ImageNet trained model correctly predicts the class `banana` with `0.95` confidence.

## 2. (Optional) Validate

Use the `classify/val.py` script to run validation for the model. This will show us the model's performance on each class.

First, we need to download ImageNet.

In [ ]:
# # WARNING: takes ~20 minutes
# !bash data/scripts/get_imagenet.sh --val

In [ ]:
# # run the validation script
# !python classify/val.py --weights ./weigths/yolov5s-cls.pt --data ../datasets/imagenet

The output shows accuracy metrics for the ImageNet validation dataset including per class accuracy.

# 3. Train On Custom Data

To train on custom data, we need to prepare a dataset with custom labels.

To prepare custom data, we'll use [Roboflow](https://roboflow.com). Roboflow enables easy dataset prep with your team, including labeling, formatting into the right export format, deploying, and active learning with a `pip` package.

If you need custom data, there are over 66M open source images from the community on [Roboflow Universe](https://universe.roboflow.com).

(For more guidance, here's a detailed blog on [training YOLOv5 classification on custom data](https://blog.roboflow.com/train-YOLOv5-classification-custom-data).)


Create a free Roboflow account, upload your data, and label.

![](https://s4.gifyu.com/images/fruit-labeling.gif)

### Load Custom Dataset

Next, we'll export our dataset into the right directory structure for training YOLOv5 classification to load into this notebook. Select the `Export` button at the top of the version page, `Folder Structure` type, and `show download code`.

The ensures all our directories are in the right format:

```
dataset
├── train
│   ├── class-one
│   │   ├── IMG_123.jpg
│   └── class-two
│       ├── IMG_456.jpg
├── valid
│   ├── class-one
│   │   ├── IMG_789.jpg
│   └── class-two
│       ├── IMG_101.jpg
├── test
│   ├── class-one
│   │   ├── IMG_121.jpg
│   └── class-two
│       ├── IMG_341.jpg
```

![](https://i.imgur.com/BF9BNR8.gif)


Copy and paste that snippet into the cell below.

In [ ]:
# Ensure we're in the right directory to download our custom dataset
import os
os.makedirs("../datasets/", exist_ok=True)
%cd ../datasets/

/content/datasets


In [ ]:
# REPLACE the below with your exported code snippet from above
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Jbx8FKHjs8dAPgHYtV7X")
project = rf.workspace("hautique-x").project("traffic-signs-detection-txg3k")
version = project.version(1)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Traffic-Signs-Detection-1 in folder:: 100%|██████████| 23726/23726 [00:02<00:00, 8114.14it/s]


In [ ]:
#Save the dataset name to the environment so we can use it in a system call later
dataset_name = dataset.location.split(os.sep)[-1]
os.environ["DATASET_NAME"] = dataset_name

### Train On Custom Data 🎉
Here, we use the DATASET_NAME environment variable to pass our dataset to the `--data` parameter.

Note: we're training for 100 epochs here. We're also starting training from the pretrained weights. Larger datasets will likely benefit from longer training.

In [ ]:
%cd ../yolov5
!python classify/train.py --model yolov5s-cls.pt --data $DATASET_NAME --epochs 100 --img 128 --pretrained weights/yolov5s-cls.pt

/content/yolov5
2024-10-20 20:40:06.632211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 20:40:06.651652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 20:40:06.658111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5s-cls.pt, data=Traffic-Signs-Detection-1, epochs=100, batch_size=64, imgsz=128, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5s-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0,

### Validate Your Custom Model

Repeat step 2 from above to test and validate your custom model.

In [ ]:
!python classify/val.py --weights runs/train-cls/exp/weights/best.pt --data ../datasets/$DATASET_NAME

classify/val: data=../datasets/Traffic-Signs-Detection-1, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-377-g24ee2801 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4221771 parameters, 0 gradients, 10.4 GFLOPs
testing:   0% 0/8 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
testing: 100% 8/8 [00:04<00:00,  1.89it/s]
                   Class      Images    top1_acc    top5_acc
                     all         983       0.995           1
              Ahead only          25           1           1
      Beware of ice-snow          20           1           1
       Bicycles crossing          22    

### Infer With Your Custom Model

In [ ]:
#Get the path of an image from the test or validation set
if os.path.exists(os.path.join(dataset.location, "test")):
  split_path = os.path.join(dataset.location, "test")
else:
  os.path.join(dataset.location, "valid")
example_class = os.listdir(split_path)[0]
example_image_name = os.listdir(os.path.join(split_path, example_class))[0]
example_image_path = os.path.join(split_path, example_class, example_image_name)
os.environ["/content/e10f94_096caabe7135431bbe50f781d504626c~mv2.jpg"] = example_image_path

print(f"Inferring on an example of the class '{example_class}'")

#Infer
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source $TEST_IMAGE_PATH

Inferring on an example of the class 'End of no passing by vehicles over 3-5 metric tons'
usage: predict.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE] [--data DATA]
                  [--imgsz IMGSZ [IMGSZ ...]] [--device DEVICE] [--view-img] [--save-txt]
                  [--nosave] [--augment] [--visualize] [--update] [--project PROJECT]
                  [--name NAME] [--exist-ok] [--half] [--dnn] [--vid-stride VID_STRIDE]
predict.py: error: unrecognized arguments: of no passing by vehicles over 3-5 metric tons/00042_00002_00005_png.rf.e108b1da0bfa0508423735410a62c80b.jpg


We can see the inference results show ~3ms inference and the respective classes predicted probabilities.

## (OPTIONAL) Improve Our Model with Active Learning

Now that we've trained our model once, we will want to continue to improve its performance. Improvement is largely dependent on improving our dataset.

We can programmatically upload example failure images back to our custom dataset based on conditions (like seeing an underrpresented class or a low confidence score) using the same `pip` package.

In [ ]:
# # Upload example image
# project.upload(image_path)


In [ ]:
# # Example upload code
# min_conf = float("inf")
# for pred in results:
#     if pred["score"] < min_conf:
#         min_conf = pred["score"]
# if min_conf < 0.4:
#     project.upload(image_path)

# (BONUS) YOLOv5 classify/predict.py Accepts Several Input Methods
- Webcam: `python classify/predict.py --weights yolov5s-cls.pt --source 0`
- Image `python classify/predict.py --weights yolov5s-cls.pt --source img.jpg`
- Video: `python classify/predict.py --weights yolov5s-cls.pt --source vid.mp4`
- Directory: `python classify/predict.py --weights yolov5s-cls.pt --source path/`
- Glob: `python classify/predict.py --weights yolov5s-cls.pt --source 'path/*.jpg'`
- YouTube: `python classify/predict.py --weights yolov5s-cls.pt --source 'https://youtu.be/Zgi9g1ksQHc'`
- RTSP, RTMP, HTTP stream: `python classify/predict.py --weights yolov5s-cls.pt --source 'rtsp://example.com/media.mp4'`

###Directory Example

In [ ]:
#Directory infer
# os.environ["TEST_CLASS_PATH"] = test_class_path = os.path.join(*os.environ["TEST_IMAGE_PATH"].split(os.sep)[:-1])
# print(f"Infering on all images from the directory {os.environ['TEST_CLASS_PATH']}")
# !python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /$TEST_CLASS_PATH/

Infering on all images from the directory content/datasets/Traffic-Signs-Detection-1/test/End of no passing by vehicles over 3-5 metric tons
usage: predict.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE] [--data DATA]
                  [--imgsz IMGSZ [IMGSZ ...]] [--device DEVICE] [--view-img] [--save-txt]
                  [--nosave] [--augment] [--visualize] [--update] [--project PROJECT]
                  [--name NAME] [--exist-ok] [--half] [--dnn] [--vid-stride VID_STRIDE]
predict.py: error: unrecognized arguments: of no passing by vehicles over 3-5 metric tons/


###YouTube Example

In [ ]:
#YouTube infer
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source 'https://www.youtube.com/watch?v=oW6m7fABIGs&pp=ygUNdHJhZmZpYyBzaWducw%3D%3D'

classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=https://www.youtube.com/watch?v=7AlYA4ItA74, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-377-g24ee2801 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4221771 parameters, 0 gradients, 10.4 GFLOPs
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

requirements: Ultralytics requirements ['pafy', 'youtube_dl==2020.12.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.5 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.2s, installed 2 packages: ['pafy', 'youtube_dl==2020.12.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

[ WARN:0@9.791] global cap.cpp:164 

**Testando o modelo**

In [55]:
import torch
import cv2
import numpy as np
from PIL import Image


In [59]:
# Caminhos para o modelo e a imagem (ajuste conforme necessário)
model_path = '/content/best.pt'  # Substitua pelo caminho do seu modelo
img_path = '/content/stop-sign.jpg'      # Stop sign
# img_path = '/content/general-caution-sign.jpg'    # General caution sign
# img_path = '/content/speed-limit-30kmh.jpg'        # Speed limit 30 km/h

In [57]:
import torch
import cv2
import numpy as np
from PIL import Image
import torch.nn.functional as F

# Carregar o modelo
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Se a GPU estiver disponível, mover o modelo para GPU
if torch.cuda.is_available():
    model.to('cuda')

# Carregar a imagem
img = Image.open(img_path)

# Converter a imagem PIL para um array NumPy
img = np.array(img)

# Redimensionar a imagem para o tamanho esperado pelo modelo (ex.: 224x224)
img = cv2.resize(img, (224, 224))

# Converter o array NumPy para um tensor PyTorch e mudar o tipo de dado para float32
img = torch.from_numpy(img).type(torch.float32)

# Normalizar a imagem (dividir os valores de pixel por 255)
img /= 255.0

# Adicionar a dimensão do lote (batch)
img = img.permute(2, 0, 1).unsqueeze(0)

# Mover a imagem para o mesmo dispositivo do modelo (GPU, se disponível)
if torch.cuda.is_available():
    img = img.to('cuda')

# Lista de classes de sinais de trânsito
classes = [
    'Ahead only', 'Beware of ice-snow', 'Bicycles crossing', 'Bumpy road',
    'Children crossing', 'Dangerous curve to the left', 'Dangerous curve to the right', 'Double curve',
    'End of all speed and passing limits', 'End of no passing', 'End of no passing by vehicles over 3-5 metric tons',
    'End of speed limit (80km-h)', 'General caution', 'Go straight or left', 'Go straight or right',
    'Keep left', 'Keep right', 'No entry', 'No passing', 'No passing for vehicles over 3-5 metric tons',
    'No vehicles', 'Pedestrians', 'Priority road', 'Right-of-way at the next intersection', 'Road narrows on the right',
    'Road work', 'Roundabout mandatory', 'Slippery road', 'Speed limit (100km-h)', 'Speed limit (120km-h)',
    'Speed limit (20km-h)', 'Speed limit (30km-h)', 'Speed limit (50km-h)', 'Speed limit (60km-h)',
    'Speed limit (70km-h)', 'Speed limit (80km-h)', 'Stop', 'Traffic signals', 'Turn left ahead', 'Turn right ahead',
    'Vehicles over 3-5 metric tons prohibited', 'Wild animals crossing', 'Yield'
]

# Realizar a classificação
results = model(img)

# Aplicar softmax para obter as probabilidades das classes
probabilities = F.softmax(results, dim=1)

# Obter o índice da classe prevista
predicted_class_idx = torch.argmax(probabilities, dim=1).item()

# Obter a confiança (probabilidade) da classe prevista
confidence = probabilities[0][predicted_class_idx].item()

# Exibir o nome da classe prevista e a confiança
predicted_class_name = classes[predicted_class_idx]
print(f'Predição de classe: {predicted_class_idx} - {predicted_class_name}')
print(f'Confiança do modelo: {confidence * 100:.2f}%')


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-377-g24ee2801 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 4221771 parameters, 0 gradients, 10.4 GFLOPs
WARNING ⚠️ YOLOv5 ClassificationModel is not yet AutoShape compatible. You must pass torch tensors in BCHW to this model, i.e. shape(1,3,224,224).


Predição de classe: 12 - General caution
Confiança do modelo: 28.53%


In [60]:
import torch
import torch.nn.functional as F

# Lista de classes de sinais de trânsito
classes = [
    'Ahead only', 'Beware of ice-snow', 'Bicycles crossing', 'Bumpy road',
    'Children crossing', 'Dangerous curve to the left', 'Dangerous curve to the right', 'Double curve',
    'End of all speed and passing limits', 'End of no passing', 'End of no passing by vehicles over 3-5 metric tons',
    'End of speed limit (80km-h)', 'General caution', 'Go straight or left', 'Go straight or right',
    'Keep left', 'Keep right', 'No entry', 'No passing', 'No passing for vehicles over 3-5 metric tons',
    'No vehicles', 'Pedestrians', 'Priority road', 'Right-of-way at the next intersection', 'Road narrows on the right',
    'Road work', 'Roundabout mandatory', 'Slippery road', 'Speed limit (100km-h)', 'Speed limit (120km-h)',
    'Speed limit (20km-h)', 'Speed limit (30km-h)', 'Speed limit (50km-h)', 'Speed limit (60km-h)',
    'Speed limit (70km-h)', 'Speed limit (80km-h)', 'Stop', 'Traffic signals', 'Turn left ahead', 'Turn right ahead',
    'Vehicles over 3-5 metric tons prohibited', 'Wild animals crossing', 'Yield'
]

# Realizar a classificação
results = model(img)

# Aplicar softmax para converter logits em probabilidades
probabilities = F.softmax(results, dim=1)

# Obter o índice da classe prevista
predicted_class_idx = torch.argmax(probabilities, dim=1).item()

# Obter a confiança da classe prevista
confidence = probabilities[0][predicted_class_idx].item()

# Exibir o nome da classe correspondente e a confiança
predicted_class_name = classes[predicted_class_idx]
print(f'Predição de classe: {predicted_class_idx} - {predicted_class_name}')
print(f'Confiança do modelo: {confidence * 100:.2f}%')


Predição de classe: 12 - General caution
Confiança do modelo: 28.53%
